In [317]:
import pandas as pd
import requests, pymongo, json, time
import os
from datetime import datetime, timedelta
from random import random
import time
# import slate

In [267]:
NYT_api_key = os.environ['NYT_API_KEY']
goodreads_key = os.environ['GOODREADS_KEY']
goodreads_secret = os.environ['GOODREADS_SECRET']


In [331]:
url = 'http://api.nytimes.com/svc/books/v2/lists/{}/combined-print-and-e-book-fiction.json&api-key={}'.format('2011-02-13', NYT_api_key)

In [332]:
cont = requests.get(url)

In [333]:
cont

<Response [200]>

In [339]:
content_data = pd.read_json(cont.content.encode('utf-8'))

In [340]:
results = content_data['results']
results[0]

{u'asterisk': 0,
 u'bestsellers_date': u'2011-01-30',
 u'book_details': [{u'age_group': u'',
   u'amazon_product_url': u'http://www.amazon.com/Tick-Tock-Michael-Bennett-Book-ebook/dp/B0047Y16MG?tag=thenewyorktim-20',
   u'author': u'James Patterson and Michael Ledwidge',
   u'book_image': u'https://s1.nyt.com/du/books/images/9780446584333.jpg',
   u'contributor': u'by James Patterson and Michael Ledwidge',
   u'contributor_note': u'',
   u'description': u'The New York detective Michael Bennett enlists the help of a former colleague to solve a rash of horrifying crimes that are throwing the city into chaos.',
   u'price': 0,
   u'primary_isbn10': u'0316037915',
   u'primary_isbn13': u'9780316037914',
   u'publisher': u'Little, Brown',
   u'title': u'TICK TOCK'}],
 u'buy_links': [{u'name': u'Local Booksellers',
   u'url': u'http://www.indiebound.org/book/9780316037914?aff=NYT'},
  {u'name': u'Barnes and Noble',
   u'url': u'http://www.anrdoezrs.net/click-7990613-11819508?url=http%3A%2F%2

In [309]:
first_results = get_book_meta(results)
first_results

{u'BODY LANGUAGE': {'author': u'Suzanne Brockmann',
  'isbn10': u'0553591657',
  'isbn13': u'9780553591651',
  'num_wks_on_list': 1,
  'rank_last_wk': 0,
  'rank_this_wk': 19},
 u'BUNGALOW 2': {'author': u'Danielle Steel',
  'isbn10': u'0440242061',
  'isbn13': u'9780440242062',
  'num_wks_on_list': 1,
  'rank_last_wk': 0,
  'rank_this_wk': 7},
 u'DARK DESIRES AFTER DUSK': {'author': u'Kresley Cole',
  'isbn10': u'0345486536',
  'isbn13': u'9780345486530',
  'num_wks_on_list': 2,
  'rank_last_wk': 18,
  'rank_this_wk': 9},
 u'HIDE': {'author': u'Lisa Gardner',
  'isbn10': u'0553588087',
  'isbn13': u'9780553588088',
  'num_wks_on_list': 1,
  'rank_last_wk': 0,
  'rank_this_wk': 4},
 u'INVISIBLE PREY': {'author': u'John Sandford',
  'isbn10': u'0425221156',
  'isbn13': u'9780425221150',
  'num_wks_on_list': 5,
  'rank_last_wk': 3,
  'rank_this_wk': 3},
 u'NATURAL BORN CHARMER': {'author': u'Susan Elizabeth Phillips',
  'isbn10': u'0060734582',
  'isbn13': u'9780060734589',
  'num_wks_on

In [159]:
content_data.head()

,copyright,last_modified,num_results,results,status
0,Copyright (c) 2016 The New York Times Company....,2016-04-28T13:06:07-04:00,35,"{u'reviews': [{u'sunday_review_link': u'', u'a...",OK
1,Copyright (c) 2016 The New York Times Company....,2016-04-28T13:06:07-04:00,35,"{u'reviews': [{u'sunday_review_link': u'', u'a...",OK
2,Copyright (c) 2016 The New York Times Company....,2016-04-28T13:06:07-04:00,35,"{u'reviews': [{u'sunday_review_link': u'', u'a...",OK
3,Copyright (c) 2016 The New York Times Company....,2016-04-28T13:06:07-04:00,35,"{u'reviews': [{u'sunday_review_link': u'', u'a...",OK
4,Copyright (c) 2016 The New York Times Company....,2016-04-28T13:06:07-04:00,35,"{u'reviews': [{u'sunday_review_link': u'', u'a...",OK


In [271]:
results[0]

{u'asterisk': 0,
 u'bestsellers_date': u'2008-05-24',
 u'book_details': [{u'age_group': u'',
   u'amazon_product_url': u'http://www.amazon.com/Hollow-Sign-Seven-Trilogy-Book/dp/0515144592?tag=thenewyorktim-20',
   u'author': u'Nora Roberts',
   u'book_image': u'https://s1.nyt.com/du/books/images/9780515144598.jpg',
   u'contributor': u'by Nora Roberts',
   u'contributor_note': u'',
   u'description': u'Book 2 in the Sign of Seven trilogy',
   u'price': 7.99,
   u'primary_isbn10': u'0515144592',
   u'primary_isbn13': u'9780515144598',
   u'publisher': u'Jove',
   u'title': u'THE HOLLOW'}],
 u'buy_links': [{u'name': u'Local Booksellers',
   u'url': u'http://www.indiebound.org/book/9780515144598?aff=NYT'},
  {u'name': u'Barnes and Noble',
   u'url': u'http://www.anrdoezrs.net/click-7990613-11819508?url=http%3A%2F%2Fwww.barnesandnoble.com%2Fw%2F%3Fean%3D9780515144598'},
  {u'name': u'Amazon',
   u'url': u'http://www.amazon.com/Hollow-Sign-Seven-Trilogy-Book/dp/0515144592?tag=thenewyorktim-

In [17]:
#Goodreads
# Get the reviews for a book given an ISBN
# Get an xml or json response that contains embed code for the iframe reviews widget that shows excerpts (first 300 characters) 
# of the most popular reviews of a book for a given ISBN. 
# The reviews are from all known editions of the book. 
# URL: https://www.goodreads.com/book/isbn/ISBN?format=FORMAT    (sample url) 
# HTTP method: GET 
# Parameters: 
# format: xml or json
# callback: function to wrap JSON response if format=json
# key: Developer key (required only for XML).
# user_id: 23298992 (required only for JSON)
# isbn: The ISBN of the book to lookup.
# rating: Show only reviews with a particular rating (optional)

# Example code for using json with callback:

#             <script type="text/javascript">
#             function myCallback(result) {
#               alert('nb of reviews for book: ' + result.reviews.length);
#             }
#             var scriptTag = document.createElement('script');
#             scriptTag.src = "https://www.goodreads.com/book/isbn/0441172717?callback=myCallback&format=json&user_id=23298992";
#             document.getElementsByTagName('head')[0].appendChild(scriptTag);
#             </script>
          


In [277]:
# Print-only (mass-market paperback, trade-paperback, hardcovers) lists dates
beginning_date = datetime(2008, 6, 8)
numdays = (datetime(2016, 11, 13) - beginning_date).days
dates = [beginning_date.strftime('%Y-%m-%d')]
for d in xrange(7, numdays + 7, 7):
    cur_date = (beginning_date + timedelta(days = d)).strftime('%Y-%m-%d')
#     cur_date = cur_date.strftime('%Y-%m-%d')
    dates.append(cur_date)
dates

['2008-06-08',
 '2008-06-15',
 '2008-06-22',
 '2008-06-29',
 '2008-07-06',
 '2008-07-13',
 '2008-07-20',
 '2008-07-27',
 '2008-08-03',
 '2008-08-10',
 '2008-08-17',
 '2008-08-24',
 '2008-08-31',
 '2008-09-07',
 '2008-09-14',
 '2008-09-21',
 '2008-09-28',
 '2008-10-05',
 '2008-10-12',
 '2008-10-19',
 '2008-10-26',
 '2008-11-02',
 '2008-11-09',
 '2008-11-16',
 '2008-11-23',
 '2008-11-30',
 '2008-12-07',
 '2008-12-14',
 '2008-12-21',
 '2008-12-28',
 '2009-01-04',
 '2009-01-11',
 '2009-01-18',
 '2009-01-25',
 '2009-02-01',
 '2009-02-08',
 '2009-02-15',
 '2009-02-22',
 '2009-03-01',
 '2009-03-08',
 '2009-03-15',
 '2009-03-22',
 '2009-03-29',
 '2009-04-05',
 '2009-04-12',
 '2009-04-19',
 '2009-04-26',
 '2009-05-03',
 '2009-05-10',
 '2009-05-17',
 '2009-05-24',
 '2009-05-31',
 '2009-06-07',
 '2009-06-14',
 '2009-06-21',
 '2009-06-28',
 '2009-07-05',
 '2009-07-12',
 '2009-07-19',
 '2009-07-26',
 '2009-08-02',
 '2009-08-09',
 '2009-08-16',
 '2009-08-23',
 '2009-08-30',
 '2009-09-06',
 '2009-09-

In [191]:
dates[0]

datetime.datetime(2008, 6, 8, 0, 0)

In [192]:
date_ = beginning_date + timedelta(days=7)

In [193]:
date_ < datetime(2011, 2, 6)

True

In [258]:
# get combined lists from 2011-02-06 to today
beg_date = datetime(2011, 2, 6)
numdays = (datetime(2016, 11, 13) - beginning_date).days
# comb_dates = [beginning_date]
# for d in xrange(0, numdays + 7, 7):
#     cur_date = beginning_date + timedelta(days = d)
# #     cur_date = cur_date.strftime('%Y-%m-%d')
# #     comb_dates.append(cur_date)
# # comb_dates
# # for list_date in comb_dates:
#     url = 'http://api.nytimes.com/svc/books/v2/lists/{}/mass-market-paperback.json&api-key={}'.format(cur_date, NYT_api_key)
#     cont = requests.get(url)
dates = [beginning_date]
for d in xrange(7, numdays + 7, 7):
    cur_date = beginning_date + timedelta(days = d)
#     cur_date = cur_date.strftime('%Y-%m-%d')
    dates.append(cur_date)
dates

[datetime.datetime(2008, 6, 8, 0, 0),
 datetime.datetime(2008, 6, 15, 0, 0),
 datetime.datetime(2008, 6, 22, 0, 0),
 datetime.datetime(2008, 6, 29, 0, 0),
 datetime.datetime(2008, 7, 6, 0, 0),
 datetime.datetime(2008, 7, 13, 0, 0),
 datetime.datetime(2008, 7, 20, 0, 0),
 datetime.datetime(2008, 7, 27, 0, 0),
 datetime.datetime(2008, 8, 3, 0, 0),
 datetime.datetime(2008, 8, 10, 0, 0),
 datetime.datetime(2008, 8, 17, 0, 0),
 datetime.datetime(2008, 8, 24, 0, 0),
 datetime.datetime(2008, 8, 31, 0, 0),
 datetime.datetime(2008, 9, 7, 0, 0),
 datetime.datetime(2008, 9, 14, 0, 0),
 datetime.datetime(2008, 9, 21, 0, 0),
 datetime.datetime(2008, 9, 28, 0, 0),
 datetime.datetime(2008, 10, 5, 0, 0),
 datetime.datetime(2008, 10, 12, 0, 0),
 datetime.datetime(2008, 10, 19, 0, 0),
 datetime.datetime(2008, 10, 26, 0, 0),
 datetime.datetime(2008, 11, 2, 0, 0),
 datetime.datetime(2008, 11, 9, 0, 0),
 datetime.datetime(2008, 11, 16, 0, 0),
 datetime.datetime(2008, 11, 23, 0, 0),
 datetime.datetime(2008,

In [ ]:
a = []
# bsl_date = results[0]['bestsellers_date']
# title = results_test[0]['book_details'][0]['title']
# isbn10 = results_test[0]['book_details'][0]['primary_isbn10']
# isbn13 = results_test[0]['book_details'][0]['primary_isbn13']

if not os.path.isfile(fname):
    a.append(entry)
    with open(fname, mode='w') as f:
        f.write(json.dumps(entry, indent=2))
else:
    with open(fname) as feedsjson:
        feeds = json.load(feedsjson)

    feeds.append(entry)
    with open(fname, mode='w') as f:
        f.write(json.dumps(feeds, indent=2))

In [215]:
url_test = 'http://api.nytimes.com/svc/books/v2/lists/{}/mass-market-paperback.json&api-key={}'.format('2008-06-13', NYT_api_key)
cont = requests.get(url_test)

In [216]:
content_data_test = pd.read_json(cont.content)

In [221]:
results_test = content_data_test['results']

In [238]:
results_test[0]['book_details'][0]

{u'age_group': u'',
 u'amazon_product_url': u'http://www.amazon.com/Hollow-Sign-Seven-Trilogy-Book/dp/0515144592?tag=thenewyorktim-20',
 u'author': u'Nora Roberts',
 u'book_image': u'https://s1.nyt.com/du/books/images/9780515144598.jpg',
 u'contributor': u'by Nora Roberts',
 u'contributor_note': u'',
 u'description': u'Book 2 in the Sign of Seven trilogy.',
 u'price': 7.99,
 u'primary_isbn10': u'0515144592',
 u'primary_isbn13': u'9780515144598',
 u'publisher': u'Jove',
 u'title': u'THE HOLLOW'}

In [292]:
results[0]['book_details'][0]['title']

u'THE HOLLOW'

In [301]:
def get_book_meta(bsl_df):
    '''
    INPUT: dataframe of json result of API call
    OUTPUT: dictionary of book meta data
    '''
    # Go thru all results and extract titles and meta data
    parsed_data = {}
    for idx, _ in enumerate(bsl_df):
        meta = {}
        title = bsl_df[idx]['book_details'][0]['title']
        author = bsl_df[idx]['book_details'][0]['author']
        meta['author'] = author
        isbn10 = bsl_df[idx]['book_details'][0]['primary_isbn10']
        meta['isbn10'] = isbn10
        isbn13 = bsl_df[idx]['book_details'][0]['primary_isbn13']
        meta['isbn13'] = isbn13
        num_wks_on_list = bsl_df[idx]['weeks_on_list']
        meta['num_wks_on_list'] = num_wks_on_list
        rank_this_wk = bsl_df[idx]['rank']
        meta['rank_this_wk'] = rank_this_wk
        rank_last_wk = bsl_df[idx]['rank_last_week']
        meta['rank_last_wk'] = rank_last_wk
        parsed[title] = meta
    return parsed_data



In [324]:
def call_api_once(date, list_name):
    '''
    INPUT: date of list, name of list, both strings
    OUTPUT: dictionary with key = date, val = parsed book meta data
    '''
    entry = {}
    url = 'http://api.nytimes.com/svc/books/v2/lists/{}/{}.json&api-key={}'.format(date, list_name, NYT_api_key)
    cont = requests.get(url)
    content_data = pd.read_json(cont.content)
    results = content_data['results']
    list_date = results[0]['bestsellers_date']
    parsed_results = get_book_meta(results)
    entry[list_date] = parsed_results
    return entry

In [347]:
def write_results_to_file(entry, filename):
    a = []
    if not os.path.isfile(filename):
        a.append(entry)
        with open(filename, mode='w') as f:
            f.write(json.dumps(entry, indent=2))
    else:
        with open(filename) as feedsjson:
            feeds = json.load(feedsjson)
#         print feeds
        feeds.update(entry)
        with open(filename, mode='w') as f:
            f.write(json.dumps(feeds, indent=2))

In [313]:
# def get_print_only_lists(dates):
#     '''
#     Goes thru all dates before NYT started including ebooks in lists and calls API on each date
#     INPUT: list of dates
#     OUTPUT: list of dictionaries in json file
#     '''
#     for date in dates:
#         entry = call_api_once(date)
#         write_results_to_file(entry, 'data/print_only_lists.json')
#         time.sleep(5 + random() * 10)
        

In [320]:
entry = {}
entry['2008-05-31'] = first_results
write_results_to_file(entry, filename = 'test/test_list.json')

In [321]:
def print_only_dates():
    # Print-only (mass-market paperback, trade-paperback, hardcovers) lists dates
    beginning_date = datetime(2008, 6, 8)
    numdays = (datetime(2016, 11, 13) - beginning_date).days
    dates = [beginning_date.strftime('%Y-%m-%d')]
    for d in xrange(7, numdays + 7, 7):
        cur_date = (beginning_date + timedelta(days = d)).strftime('%Y-%m-%d')
        dates.append(cur_date)
    return dates

In [ ]:
def get_print_only_lists():
    '''
    Goes thru all dates before NYT started including ebooks in lists and calls API on each date
    OUTPUT: list of dictionaries in json file
    '''
    dates = print_only_dates()
    for date in dates:
        hardcover_entry = call_api_once(date, list_name = 'hardcover-fiction')
        write_results_to_file(hardcover_entry, 'data/print_only_lists.json')
        time.sleep(5 + random() * 10)
        trade_paper_entry = call_api_once(date, list_name = 'trade-fiction-paperback')
        write_results_to_file(hardcover_entry, 'data/print_only_lists.json')
        time.sleep(5 + random() * 10)
        mass_paper_entry = call_api_once(date, list_name = 'mass-market-paperback')
        write_results_to_file(hardcover_entry, 'data/print_only_lists.json')
        time.sleep(5 + random() * 10)

In [334]:
def combined_list_dates():
    # combined list dates from 2/13/2011 to present
    beginning_date = datetime(2011, 2, 13)
    numdays = (datetime(2016, 11, 13) - beginning_date).days
    dates = [beginning_date.strftime('%Y-%m-%d')]
    for d in xrange(7, numdays + 7, 7):
        cur_date = (beginning_date + timedelta(days = d)).strftime('%Y-%m-%d')
        dates.append(cur_date)
    return dates

In [336]:
def get_combined_lists():
    dates = combined_list_dates()
    for date in dates:
        entry = call_api_once(date, list_name = 'combined-print-and-e-book-fiction')
        write_results_to_file(entry, 'data/combined_lists_test.json')
        time.sleep(5 + random() * 10)

In [349]:
get_combined_lists()

In [348]:
second_result = get_book_meta(results)
second_result
entry = {}
entry['2011-01-30'] = first_results
write_results_to_file(entry, filename = 'test/test_list.json')